# Digit Recognition

In [3]:
import torch
import csv
from tqdm import tqdm
from utils.get_checkpoint import get_checkpoint
from utils.model import Model
from utils.data_loader import DataLoader
from utils.evaluate import Evaluate

import yaml

with open("config.yml", "r") as f:
    data = yaml.safe_load(f)
    
TRAIN_DATA_DIR = data["TRAIN_DATA_DIR"]
TEST_DATA_DIR = data["TEST_DATA_DIR"]
LOAD_DIR = data["LOAD_DIR"]
RESULT_DIR = data["RESULT_DIR"]
SAVE_DIR = data["SAVE_DIR"]

### Parameters
· params:<br>

| Param    | Type   | Function                |
|----------|--------|-------------------------|
| train    | bool   | 设置为`True`时，模型开始训练；反之推理。 | 
| pre_trained| bool | 设置为`True`时，加载模型         |
| save_model | bool | 设置为`True`时，模型参数保存 |
| save     | bool   | 设置为`True`时，保存输出         | 
| data_dir | str    | 需要使用的数据集路径              | 
| load_dir | str    | 模型加载的路径|
| save_model_dir | str | 模型保存的路径|
| save_dir | str    | 输出保存路径                  | 

In [4]:
train = True
pre_trained = True
save_model = False
save = False

data_dir = TRAIN_DATA_DIR
load_dir = LOAD_DIR
result_dir = RESULT_DIR
model_save_dir = None

torch_float = torch.float32

· 加载模型

In [8]:
model = Model(10)
if pre_trained:
    try:
        checkpoint = get_checkpoint(load_dir).get()
        print(f"Loading checkpoint: {model.load(checkpoint)}")
    except:
        print(f"No checkpoint found at {load_dir}")

loader = DataLoader(data_dir, train=train)
data_sum = loader.__len__()
epoch = 1

Loading checkpoint: True


· 模型训练 & 推理

In [6]:
' 设置数据数量 '
data_sum = int(input(' 设置数据数量 '))
data_sum = min(loader.__len__(), data_sum)
' 设置epoch '
epoch = int(input(' 设置epoch '))

ValueError: invalid literal for int() with base 10: ''

In [14]:
pred = []
# 推理下没有label
if train:
    labels = []

if train:
    right = 0
    for _ in range(epoch):
        print(f'Epoch: {_ + 1}')
        for i in tqdm(range(data_sum)):
            data = loader.get_data()[0]
            labels.append(data[0])
            inputs = torch.tensor(data[1]).unsqueeze(0).to(torch_float)
    
            output = model(inputs)
            _, class_idx = torch.max(output, 1)
            pred.append(class_idx)
            
            if class_idx == labels:
                right += 1
        print(f"\n\tRight: {right}\t Wrong: {data_sum - right}\n\tAccuracy: {right / data_sum * 100:.2f}%\n")
        # model save
        if save_model:
            try:
                print(f'Model Saved: {model.save(model_save_dir)}')
            except:
                print(f'No file found at: {model_save_dir}')
    print("Train Finished!")
    
else:
    for _ in range(epoch):
        print(f'Epoch: {_ + 1}')
        for i in tqdm(range(data_sum)):
            data = loader.get_data()[0]
            inputs = torch.tensor(data[0]).unsqueeze(0).to(torch.float)
            output = model(inputs)
            pred.append(output)
    print("Reference Finished!")


Epoch: 1


100%|██████████| 42000/42000 [16:15<00:00, 43.06it/s]   


	Right: 0	 Wrong: 42000
	Accuracy: 0.00%

Train Finished!


· 保存结果

In [ ]:
if save:
    try:
        train_res = [['true label', 'predicted label']]
        for i in range(len(pred)):
            train_res.append([labels[i], int(pred[i])])
    
        with open(RESULT_DIR, 'w') as f:
            writer = csv.writer(f)
            writer.writerows(train_res)
            
        print(f"Train Results Saved Successfully!")
    except:
        print(f"No file found at: {RESULT_DIR}")

· 模型评估

In [ ]:
if len(pred) + len(labels):
    eval = Evaluate(pred, labels, [i + 1 for i in range(10)])
    eval.get_map()
    print(f"Accuracy: {eval.get_accuracy()}")
    print(f"Precision: {eval.get_precision()}")
    print(f"Recall: {eval.get_recall()}")
    print(f"F1 score: {eval.get_f1()}")
else:
    print("No Prediction Found!")

### 其他模型

· 加载数据

In [ ]:
from sklearn.model_selection import train_test_split
# load data
data = []
labels = []
with open(TRAIN_DATA_DIR, "r") as f:
    reader = csv.reader(f)
    next(reader)

    for row in reader:
        label = int(row[0])
        img = [int(i) for i in row[1:]]
        data.append(img)
        labels.append(label)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

· SVM尝试

In [ ]:
from other_models.SVM import SVM, findBestComponents

# find best model
findBestComponents(X_train, X_test, y_train, y_test)


· 决策树尝试

In [15]:
from other_models.DecisionTree import DicisionTree, getBestDepth

getBestDepth(X_train, X_test, y_train, y_test)

NameError: name 'X_train' is not defined